In [ ]:
!pip install --upgrade deepxde
import numpy as np
import cmath
import math
import mpmath as mp
import deepxde as dde
from deepxde.backend import tf
import scipy as sc
import functools
import matplotlib.pyplot as plt
import io
import re

import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 121 kB 4.9 MB/s 
     |████████████████████████████████| 100 kB 5.4 MB/s 


Deepxde backend not selected or invalid. Assuming tensorflow.compat.v1 for now.
Using backend: tensorflow.compat.v1



Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDEBACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax (all lowercase)
Instructions for updating:
non-resource variables are not supported in the long term



In [7]:
N = 1.0 ### Mode number
L = 2.0 ### Angular momentum multipole number

M = 1.0 # Mass of black hole
Lambda = 5.0001 # cosmological constant
gamma = 0.05
zeta = 2.51465
rh =  0.8460 ### Event Horizon
rc =  0.8509 ###

ms = 0.0
Q = 0.0 #charge
####################################### https://journals.aps.org/prd/pdf/10.1103/PhysRevD.99.084002
kappa = 1 ######## 
kappa_ = ((rc-rh)*(1 + zeta + gamma*rh - 2*(Q**2)/(rh**2)))/(2*rh**2) # kappa The surface gravity at the BH horizon (event horizon) 
V0 = ((kappa**2)*(L*(L - 1) + (ms**2)*(rh**2)))/(1 + zeta + gamma*rh - 2*(Q**2)/(rh**2)) 

In [ ]:
# our variables
omegaR = tf.Variable(1.0) ### omega = omegaR + i omegaI
omegaI = tf.Variable(-1.0) 
######
Dom_points = 100

### definig our PDE 

def pde(y, psi):
    psiR, psiI = psi[:, 0:1], psi[:, 1:2] ### psi = psiR + i psiI
    dpsiR_y = dde.grad.jacobian(psi, y, i=0)
    dpsiI_y = dde.grad.jacobian(psi, y, i=1)
    dpsiR_yy = dde.grad.hessian(psi, y, component=0, i=0, j=0)
    dpsiI_yy = dde.grad.hessian(psi, y, component=1, i=0, j=0)
    dpsiR_yyy = dde.grad.jacobian(dpsiR_yy, y, i=0)
    dpsiI_yyy = dde.grad.jacobian(dpsiI_yy, y, i=0)
    dR = ((kappa**2) * (1 - y**2)**2) * dpsiR_yy - 2 * y * (kappa**2) * (1 - y**2) * dpsiR_y- (V0 * (1 - y**2)) * psiR - 2*omegaI*omegaR*psiI + (omegaR**2 - omegaI**2)*psiR
    ddR =((kappa**2) * (1 - y**2)**2) * dpsiR_yyy - 6 * kappa**2 * y * (1 - y**2) * dpsiR_yy - ((1 - y**2) * (V0 + 2 * kappa**2) + 2 * (kappa * y)**2) * dpsiR_y + 2 * V0 * psiR - 2 * omegaI * omegaR * dpsiI_y + (omegaR**2 - omegaI**2) * dpsiR_y
    dI = ((kappa**2) * (1 - y**2)**2) * dpsiI_yy - 2 * y * (kappa**2) * (1 - y**2) * dpsiI_y - (V0*(1 - y**2)) * psiI + 2*omegaI*omegaR*psiR + (omegaR**2 - omegaI**2)*psiI
    ddI =((kappa**2) * (1 - y**2)**2) * dpsiI_yyy - 6 * kappa**2 * y * (1 - y**2) * dpsiI_yy - ((1 - y**2) * (V0 + 2 * kappa**2) + 2 * (kappa * y)**2) * dpsiI_y + 2 * V0 * psiI + 2 * omegaI * omegaR * dpsiR_y + (omegaR**2 - omegaI**2) * dpsiI_y
    return [dR, dI]

def funcR(y):### Real part of psi (true solution) https://arxiv.org/pdf/0905.2975.pdf
    PSIR = []
    for i in y:
        Xi = 1/(1 + mp.exp(-2*kappa*mp.atanh(i)))      
        Beta = ((1/4) - (V0/(kappa**2)))**(1/2)
        Omega = (kappa)*(-(N + 1/2)*1j + ((V0/(kappa**2)) - (1/4))**(1/2))
        a = 1/2 + Beta - ((Omega*1j)/kappa)
        b = 1/2 - Beta - ((Omega*1j)/kappa)
        c = 1 - ((Omega*1j)/kappa)
        z = Xi
        psiR = float(mp.re(((Xi*(1 - Xi))**((-Omega*1j)/(2*kappa)))*mp.hyp2f1(a, b, c, z)))
        PSIR.append(psiR)
    return np.array(PSIR)[:, None]

def funcI(y):### Imaginary part of psi (true solution) https://arxiv.org/pdf/0905.2975.pdf
    PSII = []
    for i in y:
        Xi = 1/(1 + mp.exp(-2*kappa*mp.atanh(i)))     
        Beta = ((1/4) - (V0/(kappa**2)))**(1/2) 
        Omega = (kappa)*(-(N + 1/2)*1j + ((V0/(kappa**2)) - (1/4))**(1/2))
        a = 1/2 + Beta - ((Omega*1j)/kappa)
        b = 1/2 - Beta - ((Omega*1j)/kappa)
        c = 1 - ((Omega*1j)/kappa)
        z = Xi  
        psiI = float(mp.im(((Xi*(1 - Xi))**((-Omega*1j)/(2*kappa)))*mp.hyp2f1(a, b, c, z)))
        PSII.append(psiI) 
    return np.array(PSII)[:, None]
  ### OTHER SOLLUTION  https://journals.aps.org/prd/pdf/10.1103/PhysRevD.99.084002

Omega = (kappa)*(-(N + 1/2)*1j + ((V0/(kappa**2)) - (1/4))**(1/2)) 
OmegaR = mp.re(Omega) ### true values (not fed into PINN, but used as reference to calculate L2 rel. error)
OmegaI = mp.im(Omega)

RE1 = ((OmegaR - omegaR)**2/OmegaR**2)**(1/2)### relative error of Re[omega]
RE2 = ((OmegaI - omegaI)**2/OmegaI**2)**(1/2)### relative error of Im[omega]
ORE = (((OmegaR - omegaR)**2 + (OmegaI - omegaI)**2)/(OmegaR**2 + OmegaI**2))**(1/2)

fnamevar = "QNF.dat" ### to save the quasinormale frequencies
variable = dde.callbacks.VariableValue([omegaR, omegaI, RE1, RE2, ORE], period=1, filename=fnamevar, precision=7)

'''the boundary conditions as usually given in the literature cannot
be applied in the code as the solution at the boundary points is infinity.'''
def boundary_l(x, on_boundary):
        return on_boundary and np.isclose(x[0], -0.7) 
def boundary_r(x, on_boundary):
        return on_boundary and np.isclose(x[0], 0.7)
geom = dde.geometry.Interval(-0.7, 0.7) ### intervalle (-0.7, 0.7) the solution at -1 and 1 is infinity
observe_y = np.linspace(-0.7, 0.7, num=100)[:, None] ### [1:] removes -1 (i.e. start point), [:, None] gives column vector form array
observe_Y = np.linspace(-0.7, 0.7, num=100)

bc1 = dde.icbc.DirichletBC(geom, lambda x: float(funcR(observe_Y)[0]), boundary_l, component=0) ## boundary conditions
bc2 = dde.icbc.DirichletBC(geom, lambda x: float(funcR(observe_Y)[-1]), boundary_r, component=0)
bc3 = dde.icbc.DirichletBC(geom, lambda x: float(funcI(observe_Y)[0]), boundary_l, component=1)
bc4 = dde.icbc.DirichletBC(geom, lambda x: float(funcI(observe_Y)[-1]), boundary_r, component=1)

PsiR = dde.PointSetBC(observe_y, funcR(observe_Y), component=0) ## generate Data set for psiR
PsiI = dde.PointSetBC(observe_y, funcI(observe_Y), component=1) ## data set psiI
data = dde.data.PDE(
        geom,
        pde,
        [bc1, bc2, bc3, bc4, PsiR, PsiI],
        num_domain=100,
        num_boundary=100,
        anchors=observe_y,       
    )

checkpoint_filepath = "/content/model/model.ckpt"
model_checkpoint_callback = dde.callbacks.VariableValue([omegaI,omegaR], period=1000, filename="QNF1.dat")

activation = "swish"
initializer = "Glorot uniform"
net = dde.nn.PFNN([1, [36, 36], [20, 20], [20, 20], 2], activation, initializer)                 
model = dde.Model(data, net)
model.compile("L-BFGS-B",loss_weights=[0.01, 0.01, 0.001, 0.001, 100, 100, 100, 100])
model.train()
learning_rate = 0.0001
model.compile("adam",lr=learning_rate, loss_weights=[0.01, 0.01, 0.001, 0.001, 100, 100, 100, 100], external_trainable_variables=[omegaI,omegaR])

epoch = 150000

losshistory, train_state = model.train(epochs = epoch, callbacks=[model_checkpoint_callback, variable]) 
#dde.saveplot(losshistory, train_state, issave=True, isplot=True)


### Plotting omega_n: PINN approximation vs. exact/true solution.
lines = open(fnamevar, "r").readlines()
omega = np.array([np.fromstring(min(re.findall(re.escape('[')+"(.*?)"+re.escape(']'),line), key=len), sep=',') for line in lines])
l,c = omega.shape
plt.plot(omega[:,0],'b-')
plt.plot(omega[:,1],'r-')
plt.plot(np.ones(omega[:,0].shape)*OmegaR,'b--')
plt.plot(np.ones(omega[:,1].shape)*OmegaI,'r--')
plt.legend([r'$\omega_R$ (approx)',r'$\omega_I$ (approx)',r'$\omega_R$ (exact)',r'$\omega_I$ (exact)'],loc = "top")
plt.title(r'$\omega_n$')
plt.xlabel('Training epoch')
plt.show()

"""L2 RO"""
plt.plot(range(l),omega[:,2],'b--')
plt.plot(range(l),omega[:,3],'r--')
plt.plot(range(l),omega[:,4],'g-')
plt.legend([r'$\omega_R$',r'$\omega_I$',r'$\omega$'],loc = "top")
plt.xlabel('Training epoch')
plt.ylabel('L2 erreur relative')
plt.title(r"Précision de $\omega_n$ (pendant l'entraînement)")
plt.show()

Compiling model...
Building feed-forward neural network...
'build' took 0.167390 s

'compile' took 8.906422 s

Initializing variables...
Training model...

Step      Train loss                                                                          Test loss                                                                           Test metric
0         [7.18e-05, 1.14e-05, 1.03e-02, 1.03e-02, 3.95e+01, 3.89e+01, 1.68e+02, 7.16e+01]    [7.18e-05, 1.14e-05, 1.03e-02, 1.03e-02, 3.95e+01, 3.89e+01, 1.68e+02, 7.16e+01]    []  
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.008751
  Number of iterations: 266
  Number of functions evaluations: 332
332       [3.79e-03, 3.00e-03, 2.23e-08, 4.59e-08, 1.80e-06, 1.50e-07, 5.86e-04, 1.37e-03]    [3.79e-03, 3.00e-03, 2.23e-08, 4.59e-08, 1.80e-06, 1.50e-07, 5.86e-04, 1.37e-03]    []  

Best model at step 332:
  train loss: 8.75e-03
  test loss: 8.75e-03
  test

In [ ]:
lines = open("QNF1.dat", "r").readlines()

omegaR_p = omega[epoch,0]*kappa_
omegaI_p = omega[epoch,1]*kappa_
OmegaR_t = OmegaR*kappa_
OmegaI_t = OmegaI*kappa_ 

R_E = (((OmegaR_t - omegaR_p)**2 + (OmegaI_t - omegaI_p)**2)/(OmegaR_t**2 + OmegaI_t**2))**(1/2)

print("******************************************|")
print("QNMs of Near Extremal dRGT BH :")
print(" N=",N," and L = ",L)
print("******************************************|")
print("PINNs")
print("******************************************|")
print("Wr = ",omega[epoch,0]*kappa_ + 1j*omega[epoch,1]*kappa_)

print("******************************************|")
print("Formula")
print("******************************************|")
print(OmegaR*kappa_ + 1j*OmegaI*kappa_)
print("******************************************|")
print("******************************************|")
print("RE = ", R_E)
print("******************************************|")
print("******************************************|")
print("kh = ", kappa_)
print("******************************************|")

IndexError: ignored